# NYC Taxi Data Exploration
## Data Engineering Zoomcamp - Module 1

**Purpose:** Understand the data before building the pipeline

**Date:** January 2026

**Author:** Ellie Pascaud

---
## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Plot settings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Setup complete")

---
## 2. Load Data

In [ ]:
# File paths
DATA_DIR = Path('../data')
TRIPS_FILE = DATA_DIR / 'green_tripdata_2025-11.parquet'
ZONES_FILE = DATA_DIR / 'taxi_zone_lookup.csv'

# Load taxi trips (parquet format)
print(f"Loading {TRIPS_FILE}...")
df_trips = pd.read_parquet(TRIPS_FILE)
print(f"✅ Loaded {len(df_trips):,} trips")

# Load zones (csv format)
print(f"\nLoading {ZONES_FILE}...")
df_zones = pd.read_csv(ZONES_FILE)
print(f"✅ Loaded {len(df_zones):,} zones")

---
## 3. Data Overview

In [ ]:
# Trips data structure
print("📊 TRIPS DATA STRUCTURE")
print("=" * 60)
print(f"Shape: {df_trips.shape}")
print(f"\nColumns: {list(df_trips.columns)}")
print(f"\nData types:")
print(df_trips.dtypes)

In [ ]:
# First rows
print("\n📋 First 5 rows:")
df_trips.head()

In [ ]:
# Statistical summary
print("📈 STATISTICAL SUMMARY")
print("=" * 60)
df_trips.describe()

---
## 4. Data Quality Checks

In [ ]:
# Missing values
print("🔍 MISSING VALUES")
print("=" * 60)
missing = df_trips.isnull().sum()
missing_pct = (missing / len(df_trips) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing': missing,
    'Percentage': missing_pct
})
print(missing_df[missing_df['Missing'] > 0])

In [ ]:
# Duplicate rows
duplicates = df_trips.duplicated().sum()
print(f"\n🔍 Duplicate rows: {duplicates:,}")

In [ ]:
# Data anomalies
print("\n⚠️ DATA ANOMALIES")
print("=" * 60)

# Negative values
neg_distance = (df_trips['trip_distance'] < 0).sum()
neg_amount = (df_trips['total_amount'] < 0).sum()
print(f"Negative trip_distance: {neg_distance:,}")
print(f"Negative total_amount: {neg_amount:,}")

# Extreme values
extreme_distance = (df_trips['trip_distance'] > 100).sum()
extreme_amount = (df_trips['total_amount'] > 500).sum()
print(f"\nTrips > 100 miles: {extreme_distance:,}")
print(f"Amounts > $500: {extreme_amount:,}")

---
## 5. Business Insights

In [ ]:
# Trip distance distribution
print("🚕 TRIP DISTANCE ANALYSIS")
print("=" * 60)

# Filter reasonable distances (< 100 miles)
reasonable_trips = df_trips[df_trips['trip_distance'] < 100]

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(reasonable_trips['trip_distance'], bins=50, edgecolor='black')
axes[0].set_xlabel('Trip Distance (miles)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Trip Distance Distribution')
axes[0].axvline(reasonable_trips['trip_distance'].median(), 
                color='red', linestyle='--', label=f'Median: {reasonable_trips["trip_distance"].median():.2f}')
axes[0].legend()

# Box plot
axes[1].boxplot(reasonable_trips['trip_distance'])
axes[1].set_ylabel('Trip Distance (miles)')
axes[1].set_title('Trip Distance Box Plot')

plt.tight_layout()
plt.show()

print(f"\nMedian distance: {df_trips['trip_distance'].median():.2f} miles")
print(f"Mean distance: {df_trips['trip_distance'].mean():.2f} miles")

In [ ]:
# Payment analysis
print("\n💳 PAYMENT ANALYSIS")
print("=" * 60)

# Total amount distribution
reasonable_amounts = df_trips[df_trips['total_amount'] < 200]

plt.figure(figsize=(12, 5))
plt.hist(reasonable_amounts['total_amount'], bins=50, edgecolor='black')
plt.xlabel('Total Amount ($)')
plt.ylabel('Frequency')
plt.title('Total Amount Distribution (< $200)')
plt.axvline(df_trips['total_amount'].median(), 
            color='red', linestyle='--', 
            label=f'Median: ${df_trips["total_amount"].median():.2f}')
plt.legend()
plt.show()

print(f"\nMedian amount: ${df_trips['total_amount'].median():.2f}")
print(f"Mean amount: ${df_trips['total_amount'].mean():.2f}")

In [ ]:
# Temporal analysis
print("\n📅 TEMPORAL ANALYSIS")
print("=" * 60)

# Trips by day
df_trips['pickup_date'] = pd.to_datetime(df_trips['lpep_pickup_datetime']).dt.date
trips_by_day = df_trips.groupby('pickup_date').size()

plt.figure(figsize=(15, 5))
trips_by_day.plot(kind='bar')
plt.xlabel('Date')
plt.ylabel('Number of Trips')
plt.title('Trips per Day (November 2025)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"\nBusiest day: {trips_by_day.idxmax()} ({trips_by_day.max():,} trips)")
print(f"Quietest day: {trips_by_day.idxmin()} ({trips_by_day.min():,} trips)")

---
## 6. Zone Analysis

In [ ]:
# Zones overview
print("🗺️ ZONES DATA")
print("=" * 60)
print(f"Total zones: {len(df_zones)}")
print(f"\nColumns: {list(df_zones.columns)}")
print("\nFirst 10 zones:")
df_zones.head(10)

In [ ]:
# Top pickup locations
print("\n🚖 TOP PICKUP LOCATIONS")
print("=" * 60)

top_pickups = df_trips['PULocationID'].value_counts().head(10)

# Merge with zone names
top_pickup_zones = df_zones[df_zones['LocationID'].isin(top_pickups.index)]
top_pickup_df = pd.DataFrame({
    'Zone': top_pickup_zones.set_index('LocationID').loc[top_pickups.index, 'Zone'].values,
    'Trips': top_pickups.values
})

print(top_pickup_df)

# Visualize
plt.figure(figsize=(12, 6))
plt.barh(top_pickup_df['Zone'], top_pickup_df['Trips'])
plt.xlabel('Number of Trips')
plt.title('Top 10 Pickup Locations')
plt.tight_layout()
plt.show()

---
## 7. Homework Questions Preview

In [ ]:
# Question 3: Trips with distance <= 1 mile
print("❓ QUESTION 3 PREVIEW")
print("=" * 60)
short_trips = len(df_trips[df_trips['trip_distance'] <= 1])
print(f"Trips with distance ≤ 1 mile: {short_trips:,}")
print(f"Percentage: {short_trips / len(df_trips) * 100:.2f}%")

In [ ]:
# Question 4: Day with longest trip
print("\n❓ QUESTION 4 PREVIEW")
print("=" * 60)
reasonable = df_trips[df_trips['trip_distance'] < 100]
longest_by_day = reasonable.groupby('pickup_date')['trip_distance'].max()
longest_day = longest_by_day.idxmax()
longest_distance = longest_by_day.max()
print(f"Day with longest trip: {longest_day}")
print(f"Distance: {longest_distance:.2f} miles")

---
## 8. Key Findings & Recommendations

### 📊 Summary Statistics

**Trips:**
- Total trips: ~58,000
- Date range: November 2025
- Median distance: ~X miles
- Median fare: $X

**Data Quality:**
- Missing values: None/Minimal
- Duplicates: None
- Anomalies: Some extreme values (>100 miles, >$500)

### 🎯 Recommendations for Pipeline

1. ✅ **Normalize column names** (already implemented)
   - PULocationID → pulocationid
   - Avoid SQL quoting issues

2. ✅ **Data validation** (to implement)
   - Filter extreme values
   - Check for negative amounts
   - Validate date ranges

3. ✅ **Indexing** (for query performance)
   - Index on pickup_datetime
   - Index on LocationIDs

4. ✅ **Chunked loading** (already implemented)
   - 10,000 rows per chunk
   - Prevents memory issues

---
## 9. Next Steps

1. ✅ Run the ingestion pipeline: `python pipeline/ingest_pipeline.py`
2. ✅ Verify data in PostgreSQL
3. ✅ Execute homework SQL queries
4. ✅ Document findings in README.md